In [5]:
%matplotlib widget

from typing import Tuple, Any, Optional, Union, Dict, List
from collections import OrderedDict
from dataclasses import dataclass

import numpy as np
from matplotlib import pyplot as plt
import lmfit

# Idea and basic use

At its heart an "Analysis" is some sort of routine that has data as input, and some number of parameters as output. 
We'll therefore be able to break down everything in terms of a few different types of basic objects:

- `Analysis`: describes the routine we subject the data to.
- `AnalysisResult`: the output of the analysis. Will always contain at least output parameters, plus any other information/methods that might be specific to the type of analysis we perform.
- `Parameter`: output parameters that contain values plus some metadata.

What we'd like is to be able to use the analysis in the following way:

```python
>>> result = analyze(MyAnalysis, coordinates, data)
>>> print(result.parameters['some_output'].value)
... 3.14
```

It should not matter at all what kind of analysis we run -- the access of output can always be uniform. 
The result of some analysis types might be more complex.
For example, a fit result can contain a function that allows accessing the best fit, and its parameters might include uncertainties.

# Prototyping

## Basic elements

The goal is to have base classes that all types of analysis can use. 
More complex analysis types will extend the basics ones.
For parameters we'll build on the idea of `Parameter` and `Parameters` in `lmfit`.
We want out versions largely compatible with the lmfit ones to be able to convert quickly.

In [6]:
class Parameter:
    
    def __init__(self, name, value: Any = None, **kw: Any):
        self.name = name
        self.value = value
        self._attrs = {}
        for k, v in kw:
            self._attrs[k] = v
            
    def __getattr__(self, key):
        return self._attrs[k]


class Parameters(OrderedDict):
    """A collection of parameters"""
    
    def add(self, name: str, **kw: Any):
        """Add/overwrite a parameter in the collection."""
        self[name] = Parameter(name, **kw)


class AnalysisResult(object):
    
    def __init__(self, parameters: Dict[str, Union[Dict[str, Any], Any]]):
        self.params = Parameters()
        for k, v in parameters.items():
            if isinstance(v, dict):
                self.params.add(k, **v)
            else:
                self.params.add(k, value=v)
                
    def eval(self, *args: Any, **kwargs: Any) -> np.ndarray:
        """Analysis types that produce data (like filters or fits) should implement this.
        """
        raise NotImplementedError
    

class Analysis(object):
    """Basic analysis object.
    
    Parameters
    ----------
    coordinates
        may be a single 1d numpy array (for a single coordinate) or a tuple
        of 1d arrays (for multiple coordinates).
    data
        a 1d array of data
    """
    
    def __init__(self, coordinates: Union[Tuple[np.ndarray, ...], np.ndarray], data: np.ndarray):
        """Constructor of `Analysis`. """
        self.coordinates = coordinates
        self.data = data
        
    def analyze(self, coordinates, data, *args: Any, **kwargs: Any) -> AnalysisResult:
        """Needs to be implemented by each inheriting class."""
        raise NotImplementedError
    
    def run(self, *args: Any, **kwargs: Any) -> AnalysisResult:
        return self.analyze(self.coordinates, self.data, **kwargs)
    
    
# def analyze(analysis_class: Analysis, coordinates: Union[Tuple[np.ndarray, ...], np.ndarray], 
#             data: np.ndarray, **kwarg: Any) -> AnalysisResult:
#     analysis = analysis_class(coordinates, data)
#     return analysis.run(**kwarg)
    
    
class FindMax(Analysis):
    """A simple example class to illustrate the concept."""
    
    def analyze(self, xvals, yvals):
        i = np.argmax(yvals)
        
        return AnalysisResult(
            dict(
                max_val=yvals[i],
                max_pos=xvals[i]
            )
        )

In [10]:
x = np.arange(10)
y = np.random.rand(10)
print(y)

analysis = FindMax(x, y)
result = analysis.run()

print(result.params['max_pos'].value, result.params['max_val'].value)

[0.01029839 0.3182288  0.45905976 0.17431109 0.32107715 0.78818021
 0.95962545 0.04488793 0.25833699 0.45830823]
6 0.9596254515651905


## Fitting

Fitting is now simply a specific type of analysis.

In [66]:
class FitResult(AnalysisResult):
    
    def __init__(self, lmfit_result: lmfit.model.ModelResult):
        self.lmfit_result = lmfit_result
        self.params = lmfit_result.params
        
    def eval(self, *args: Any, **kwargs: Any):
        return self.lmfit_result.eval(*args, **kwargs)


class Fit(Analysis):
    
    @staticmethod
    def model(*arg, **kwarg) -> np.ndarray:
        raise NotImplementedError
           
    def analyze(self, coordinates, data, dry=False, params={}, **fit_kwargs):
        model = lmfit.model.Model(self.model)
        
        _params = lmfit.Parameters()
        for pn, pv in self.guess(coordinates, data).items():
            _params.add(pn, value=pv)
        for pn, pv in params:
            _params[pn] = pv
        
        if dry:
            lmfit_result = lmfit.model.ModelResult(model, params=_params, 
                                                   data=data, coordinates=coordinates)
        else:
            lmfit_result = model.fit(data, params=_params, 
                                     coordinates=coordinates, **fit_kwargs)
        
        return FitResult(lmfit_result)
    
    @staticmethod
    def guess(self, coordinates, data) -> Dict[str, Any]:
        raise NotImplementedError
        
        
class CosineFit(Fit):
    
    @staticmethod
    def model(coordinates, A, f, phi, of) -> np.ndarray:
        """$A \cos(2 \pi f x + \phi) + of$"""
        return A * np.cos(2 * np.pi * coordinates * f + phi) + of
    
    @staticmethod
    def guess(coordinates, data):
        of = np.mean(data)
        A = (np.max(data) - np.min(data)) / 2.
        
        fft_val = np.fft.rfft(data)[1:]
        fft_frq = np.fft.rfftfreq(data.size, np.mean(coordinates[1:]-coordinates[:-1]))[1:]
        idx = np.argmax(np.abs(fft_val))
        f = fft_frq[idx]
        phi = np.angle(fft_val[idx])
        
        return dict(A=A, of=of, f=f, phi=phi)

In [67]:
tvals = np.linspace(0, 2, 51)
data = 1.5 * np.cos(2*np.pi*1 * tvals + np.pi) + 0.5 + np.random.normal(scale=0.2, size=tvals.size)

fit = CosineFit(tvals, data)
fit_guess = fit.run(dry=True)
fit_result = fit.run()

fig, ax = plt.subplots(1,1)
ax.plot(tvals, data, 'o', label='data')
ax.plot(tvals, fit_guess.eval(coordinates=tvals), '--', zorder=-1, label='guess')
ax.plot(tvals, fit_result.eval(coordiantes=tvals), '-', label='fit to: ' + fit.model.__doc__, lw=2)
ax.legend(loc=0, fontsize='small')

print(fit_result.lmfit_result.fit_report())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    Model(model)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 26
    # data points      = 51
    # variables        = 4
    chi-square         = 1.99778913
    reduced chi-square = 0.04250615
    Akaike info crit   = -157.229009
    Bayesian info crit = -149.501707
[[Variables]]
    A:    1.54433140 +/- 0.04099138 (2.65%) (init = 1.751884)
    of:   0.53002359 +/- 0.03143838 (5.93%) (init = 0.5102376)
    f:    0.99311124 +/- 0.00816438 (0.82%) (init = 0.9803922)
    phi: -3.14572689 +/- 0.05812046 (1.85%) (init = -3.063049)
[[Correlations]] (unreported correlations are < 0.100)
    C(f, phi)  = -0.889
    C(of, f)   =  0.396
    C(of, phi) = -0.351
    C(A, f)    = -0.142
    C(A, phi)  =  0.126
